<a href="https://colab.research.google.com/github/albertomors/Bachelor_Thesis_E2207/blob/main/ORION_ARCH_v0.5_per_32x32x32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import h5py

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
data_dir = "/content/drive/My Drive/Dataset_download/"
save_path = "/content/drive/My Drive/pth_results/"
# Assuming VoxelDatasetHDF5 is already defined as in the previous code snippet
hdf5_file_path = "/content/drive/My Drive/output_dataset.h5"

ORIENT_LABELS = 18

if torch.cuda.is_available():
    print("GPU is available!")
    device = 'cuda'
else:
    print("Using CPU.")
    device = 'cpu'

GPU is available!


In [13]:
class VoxelDatasetHDF5(Dataset):
    def __init__(self, hdf5_file, train=True, transform=None):
        self.hdf5_file = hdf5_file
        self.train = train
        self.transform = transform

        # Open the HDF5 file
        with h5py.File(self.hdf5_file, 'r') as f:
            # Get class names from the structure of the HDF5 file
            self.classes = sorted(list(f['classes'].keys()))  # Assuming top-level is 'classes'

            # Collect all paths (keys) to the datasets inside HDF5
            self.file_paths = []
            for class_name in self.classes:
                folder = 'train' if train else 'test'
                class_group = f['classes'][class_name][folder]
                self.file_paths += [(class_name, folder, file_name) for file_name in class_group.keys()]

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        class_name, folder, file_name = self.file_paths[idx]

        # Detect corrupted files based on the presence of "(1)" in the file name
        if "(1)" in file_name:
            print(f"Corrupted file detected and skipped: {file_name}")
            return None  # Skip this file

        # Load voxelized numpy array from the HDF5 file
        with h5py.File(self.hdf5_file, 'r') as f:
            voxel_data = f['classes'][class_name][folder][file_name][()]

        # Add a channel dimension: Shape (1, 33, 33, 33)
        voxel_data = np.expand_dims(voxel_data, axis=0)

        if self.transform:
            voxel_data = self.transform(voxel_data)

        voxel_data = torch.tensor(voxel_data, dtype=torch.float32)

        # Get class label
        class_label = self.classes.index(class_name)

        # Extract the orientation from the file name and convert to a bin index based on the number of bins for this class
        try:
            orientation_number = int(file_name.split('_')[-1].split('.')[0])
        except:
            orientation_number = 17  # Handle error case
            print(f"Error parsing orientation for file: {file_name}")

        orientation_label = torch.tensor(orientation_number, dtype=torch.long)

        return voxel_data, class_label, orientation_label

class ImprovedORION(nn.Module):
    def __init__(self, num_class=40, num_orient=105):
        super().__init__()

        self.conv_net = nn.Sequential(
            #conv1
            nn.Conv3d(1,32,3,2,0), #1x 33^3 -> 32x 16^3
            nn.BatchNorm3d(32),
            nn.LeakyReLU(0.1,True),
            nn.Dropout3d(0.2),

            #conv2
            nn.Conv3d(32,64,3,1,0), #32x 16^3 -> 64x 14^3
            nn.BatchNorm3d(64),
            nn.LeakyReLU(0.1,True),
            nn.Dropout3d(0.3),

            #conv3
            nn.Conv3d(64,128,3,1,0), #64x 14^3 -> 128x 12^3
            nn.BatchNorm3d(128),
            nn.LeakyReLU(0.1,True),
            nn.Dropout3d(0.4),

            #conv4
            nn.Conv3d(128,256,3,1,0), #128x 12^3 -> 256x 10^3
            nn.BatchNorm3d(256),
            nn.LeakyReLU(0.1,True),

            #pooling
            nn.MaxPool3d(2,2), #256x 10^3 -> 256x 5^3
            nn.Dropout3d(0.6),

            #NEW LAYER
            nn.Conv3d(256,128,3,1,0), #256x 5^3 -> 128x 3^3
            nn.BatchNorm3d(128),
            nn.LeakyReLU(0.1,True),
            nn.Dropout3d(0.7),
        )

        self.flatten = nn.Flatten() #128x 3^3 -> 3456

        self.fc_net = nn.Sequential(
            nn.Linear(128*3**3, 256), #3456 -> 256
            nn.LeakyReLU(0.1, inplace=True),
            nn.Linear(256, num_class + num_orient), #256 -> "40+105" (for simplicity)
        )
        # Initialize the weights
        self.num_class = num_class
        self.num_orient = num_orient

    def forward(self, x):
        x = self.conv_net(x)
        x = self.flatten(x)
        x = self.fc_net(x)

        # Split the output into class and orientation
        class_output = x[:, :self.num_class]  # First part for class
        orient_output = x[:, self.num_class:]  # Remaining part for orientation
        return class_output, orient_output


class CustomCrossEntropyLoss(nn.Module):
    def __init__(self, neurons_per_class, gamma=0.5):
        super(CustomCrossEntropyLoss, self).__init__()
        self.neurons_per_class = neurons_per_class

        self.incremental = np.cumsum(self.neurons_per_class)
        self.incremental = np.insert(self.incremental, 0, 0)
        self.incremental = self.incremental[:-1]
        #se i neuroni per classe sono:  [6 3 6 6  6  9  3  6  1  9]
        #gli starting points sono:      [0 6 9 15 21 27 36 39 45 46]

        self.scaling_factor = np.array([ ORIENT_LABELS // self.neurons_per_class[i] for i in range(len(self.neurons_per_class))])
        #di quanto devo dividere i label per ottenere il neurone corretto
        #se i neuroni per le classi 0,1 sono: [6     3]
        #lo scaling factor è:                 [18/6  18/3]

        print("\nNEURONI PER CLASSE:", self.neurons_per_class)
        print("INCREMENTALI:", self.incremental,"\n")

        self.cross_entropy = nn.CrossEntropyLoss()
        self.gamma = gamma

    #                     10/40,          105,         1-10/40         1-18
    def forward(self, class_neurons, orient_neurons, class_labels, orient_labels):

        #print("labels = ", orient_labels)
        #remap the labels
        for i in range(orient_labels.shape[0]):
            orient_labels[i] = self.incremental[class_labels[i]] + orient_labels[i] // self.scaling_factor[class_labels[i]]
            #sposto il label sullo starting point per la classe  + offset relativo in base al numero di neuroni
        #print("remapped labels = ", orient_labels)

        class_loss = self.cross_entropy(class_neurons, class_labels)
        #print("class_loss = ", class_loss)
        orient_loss = self.cross_entropy(orient_neurons, orient_labels)
        #print("orient_loss = ", orient_loss)

        # Combine the losses
        total_loss = self.gamma * class_loss + (1-self.gamma) * orient_loss
        print(total_loss)
        return total_loss


# Define your training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, device=device):
    model.to(device)

    #dont train the loss
    for param in criterion.parameters():
          param.requires_grad = False

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # Loop over the dataset
        for voxel_data, class_labels, orientation_labels in tqdm(train_loader):
            # Move data to the device
            voxel_data = voxel_data.to(device)
            class_labels = class_labels.to(device)
            orientation_labels = orientation_labels.to(device)

            # Forward pass
            optimizer.zero_grad()
            class_outputs, orient_outputs = model(voxel_data)

            # Compute loss
            loss = criterion(class_outputs, orient_outputs, class_labels, orientation_labels)

            # Backpropagation and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print statistics at the end of every epoch
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
        save_model(epoch, model, optimizer, loss.item(), save_path=save_path)
        # Save the model after each epoch
        print(f"Model saved for epoch {epoch} at model_epoch_{epoch}.pth")

def save_model(epoch, model, optimizer, loss, save_path=save_path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, save_path)
    print(f"Model saved to {save_path}")

def load_checkpoint(model, optimizer, checkpoint_path="model_checkpoint.pth"):
    checkpoint = torch.load(checkpoint_path)

    # Load model and optimizer state dicts
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    print(f"Checkpoint loaded. Resuming from epoch {epoch}, with loss {loss}")

    return model, optimizer, epoch, loss

"""
checkpoint = torch.load("model_epoch_X.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
"""

'\ncheckpoint = torch.load("model_epoch_X.pth")\nmodel.load_state_dict(checkpoint[\'model_state_dict\'])\noptimizer.load_state_dict(checkpoint[\'optimizer_state_dict\'])\nepoch = checkpoint[\'epoch\']\nloss = checkpoint[\'loss\']\n'

In [21]:
#prova model
input_batch = torch.randint(0, 2, size=(64, 1, 33, 33, 33), dtype=torch.float32)
input_batch.size
model = ImprovedORION(num_orient = 190)
class_outputs, orient_outputs = model(input_batch)
print(class_outputs.shape)
print(orient_outputs.shape)
print(class_outputs[0])
print(orient_outputs[0])

torch.Size([64, 40])
torch.Size([64, 190])
tensor([-0.3699,  0.1259, -0.1307, -0.3566, -0.2611, -0.3537,  0.3624,  0.5427,
         0.1242, -0.0222, -0.5171, -0.1341, -0.6623,  0.0412, -0.4195,  0.0300,
         0.0216,  0.1565,  0.2650, -0.0801, -0.4885,  0.5263, -0.2889, -0.0111,
        -0.0154,  0.0836, -0.2242, -0.1545,  0.2030, -0.0706,  0.0311, -0.1958,
         0.1486, -0.2081, -0.1979, -0.1515, -0.1328, -0.2449, -0.6668, -0.1252],
       grad_fn=<SelectBackward0>)
tensor([-0.2600,  0.2303,  0.1305,  0.0117, -0.0155,  0.4626, -0.3995, -0.0498,
         0.1923,  0.3847,  0.0395, -0.1001, -0.2890, -0.2283, -0.0256, -0.1845,
        -0.0068,  0.3179, -0.0694,  0.1707, -0.2580,  0.1130, -0.0796, -0.2386,
         0.2441, -1.0539, -0.2291,  0.5037, -0.1704,  0.2650,  0.0044,  0.0592,
         0.1292,  0.3306,  0.4639,  0.1294, -0.0150,  0.4087, -0.2726,  0.4168,
        -0.5595,  0.0672,  0.5712,  0.0817,  0.0691, -0.6797,  0.0889, -0.0194,
        -0.2340,  0.3094,  0.1349,  0.0185

In [22]:
# Initialize dataset
train_dataset = VoxelDatasetHDF5(hdf5_file=hdf5_file_path, train=True)
# Create a DataLoader with the dataset
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

In [23]:
#ModelNet10
#neurons_per_class = np.array([6, 3, 6, 6, 6, 9, 3, 6, 1, 9])

#ModelNet40
neurons_per_class = np.array([6, 6, 1, 6, 6, 1, 1, 3, 6, 1, 3, 9, 6, 6, 6, 3, 3, 9, 1, 3, 6, 6, 9, 3, 6, 9, 6, 3, 6, 6, 6, 9, 3, 1, 3, 6, 3, 1, 6, 6])

# Initialize the model, loss function, and optimizer
num_class = len(train_dataset.classes)
num_orient = neurons_per_class.sum()  # Customize based on your setup
print(num_class)
print(num_orient)

40
190


In [ ]:
model = ImprovedORION(num_class=num_class, num_orient=num_orient)
criterion = CustomCrossEntropyLoss(neurons_per_class=neurons_per_class)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Assuming 'model' and 'optimizer' are already defined
#model, optimizer, start_epoch, last_loss = load_checkpoint(model, optimizer, "model_epoch_X.pth")

# You can now continue training from 'start_epoch'
train_model(model, train_loader, criterion, optimizer, num_epochs=10, device=device)


NEURONI PER CLASSE: [6 6 1 6 6 1 1 3 6 1 3 9 6 6 6 3 3 9 1 3 6 6 9 3 6 9 6 3 6 6 6 9 3 1 3 6 3
 1 6 6]
INCREMENTALI: [  0   6  12  13  19  25  26  27  30  36  37  40  49  55  61  67  70  73
  82  83  86  92  98 107 110 116 125 131 134 140 146 152 161 164 165 168
 174 177 178 184] 



  0%|          | 0/2666 [00:00<?, ?it/s]

tensor(4.5487, device='cuda:0', grad_fn=<AddBackward0>)


  0%|          | 2/2666 [00:01<30:53,  1.44it/s]

tensor(4.4568, device='cuda:0', grad_fn=<AddBackward0>)


...

tensor(1.4375, device='cuda:0', grad_fn=<AddBackward0>)


 29%|██▉       | 767/2666 [04:22<12:12,  2.59it/s]

tensor(1.5132, device='cuda:0', grad_fn=<AddBackward0>)


 29%|██▉       | 768/2666 [04:23<13:42,  2.31it/s]

tensor(1.5448, device='cuda:0', grad_fn=<AddBackward0>)
